In [1]:
import pandas as pd
from convokit import Corpus, Utterance, Conversation
import os
os.listdir()

['ydata-ynacc-v1_0',
 'ynacc-turk-conversion.ipynb',
 '.ipynb_checkpoints',
 'ynacc-expert-conversion.ipynb']

In [2]:
os.chdir('ydata-ynacc-v1_0')

In [5]:
for filedir in os.listdir():
    print(filedir)

ydata-ynacc-v1_0_turk_annotations.tsv
ydata-ynacc-v1_0_test-ids.txt
ydata-ynacc-v1_0_IAC_annotations.tsv
ydata-ynacc-v1_0_unlabeled_conversations.tsv
ydata-ynacc-v1_0_dev-ids.txt
basic-version
WebscopeReadme.txt
expert-ynacc
ydata-ynacc-v1_0_expert_annotations.tsv
README.txt
ydata-ynacc-v1_0_parentid_category.tsv
ydata-ynacc-v1_0_unlabeled_conversations.csv
ydata-ynacc-v1_0_train-ids.txt


In [13]:
# basic_data = pd.read_csv('ydata-ynacc-v1_0_unlabeled_conversations.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
annot = pd.read_csv('ydata-ynacc-v1_0_turk_annotations.tsv', sep='\t')

In [8]:
annot.head()

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type
0,50735,0,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,QcJlZUh9ahUFn0EHRgaS,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,1459625991,21,13,"OMG, I should shoot myself for voting for her ...",1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
1,50735,1,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,IYp4~4xH1otqco8_0iDq,00002g000000000000000000000000-1d5d6112-78c5-4...,1459626067,4,5,yep,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
2,50735,2,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,T_rh7XPzhe8SgWNLJx1N,00003n000000000000000000000000-dd89dec7-63cd-4...,1459626261,1,1,you got it what a human or animal,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
3,50735,3,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,OgLSacyNcC9ECBmF5yWs,00004b000000000000000000000000-e377ed85-e972-4...,1459626297,2,2,just wing yourself..a flesh wound will be fine,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
4,50735,4,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,aNYk4ScmUVFLZ_NG_0RF,00005g000000000000000000000000-b6a381b1-8c11-4...,1459626334,0,2,Nope!,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar


In [10]:
from convokit import Speaker

In [11]:
speakers = {x: Speaker(id=x) for x in annot['guid']}

In [12]:
annot[(annot.sdid==53971) & (annot.commentindex==2-1)]['commentid'].iloc[0]

'00002n000000000000000000000000-1c30b878-b717-4e9a-9872-2ce2906ce783'

In [13]:
import math

In [130]:
utts_data = dict()
for row in annot.iterrows():
    data = row[1]
    if data.commentindex == 0:
        reply_to = None
    else:
        try:
            subset = annot[(annot.sdid==data.sdid)]
            reply_to = subset[subset.commentindex==data.commentindex-1]['commentid'].iloc[0]
        except IndexError:
            print(f"{data.sdid} {data.commentindex}")
    if isinstance(data.parentid, float) and math.isnan(data.parentid):
        convo_id = data.commentid
    else:
        convo_id = data.parentid
    utt_data = {'sdid': data.sdid,
                'headline': data.headline,
                'url': data.url,
                'speaker': speakers[data.guid],
                'utt_id': data.commentid,
                'timestamp': data.timestamp,
                'text': data.text,
                'reply_to': reply_to,
                'conversation_id': convo_id
#                 'thumbsup': data['thumbs-up'],
#                 'thumbsdown': data['thumbs-down']
               }
    utts_data[data.commentid] = utt_data

90808 3
90808 3
90808 3


In [131]:
len(utts_data)

9160

The conversation at sdid 90808 is incomplete.

In [132]:
utts = []
for utt_id, utt_data in utts_data.items():
    if utt_data['sdid'] == 90808: continue
    utt = Utterance(id=utt_data['utt_id'], speaker=utt_data['speaker'], timestamp=utt_data['timestamp'],
                   text=utt_data['text'], reply_to=utt_data['reply_to'], conversation_id=utt_data['conversation_id'])
    utts.append(utt)

In [133]:
from convokit import Corpus

In [134]:
corpus = Corpus(utterances=utts)

In [135]:
corpus.print_summary_stats()

Number of Speakers: 6402
Number of Utterances: 9156
Number of Conversations: 1399


In [136]:
for convo in corpus.iter_conversations():
    if not convo.check_integrity(verbose=False):
        print(convo.id)

In [148]:
corpus.random_conversation().print_conversation_structure(lambda utt: utt.speaker.id + "\n")

UpzTRElEU_07fqw_aNu3

    1DydgeuoctOz82hkxudm

        UpzTRElEU_07fqw_aNu3

            1DydgeuoctOz82hkxudm

                UpzTRElEU_07fqw_aNu3

                    1DydgeuoctOz82hkxudm



In [145]:
corpus.dump('basic-version', base_path='.')

In [146]:
corpus.print_summary_stats()

Number of Speakers: 6402
Number of Utterances: 9156
Number of Conversations: 1399


## Adding metadata

In [193]:
corpus = Corpus('basic-version')

In [194]:
for convo in corpus.iter_conversations():
    data = utts_data[convo.id]
    keys = ['url', 'headline', 'sdid']
    for k in keys:
        convo.add_meta(k, data[k])

In [195]:
corpus.random_conversation().meta

{'url': 'https://tvline.com/2016/04/05/michael-weatherly-leaving-ncis-storyline-dinozzo-exit/',
 'headline': 'Michael Weatherly Leaving NCIS: Did We Just Find Out How Tony Will Exit?',
 'sdid': 125956}

In [196]:
corpus.random_utterance().id

'00005n000000000000000000000000-d081cf91-59dd-4435-b89d-fc6c7ae822b0'

     1  sdid - subdialogue ID
     2  commentindex - position of the comment as it appears in the subdialogue
     3  headline - headline of the article
     4  url - URL of the original article
     5  guid  - encrypted user id
     6  commentid - comment id
     7  timestamp - timestamp of the comment
     8  thumbs-up - comment level thumbsup
     9  thumbs-down - comment level thumbsdown
     10 text  - text of the comment
     11 parentid - ID associated with the first comment of the subdialogue
     12 constructiveclass - Constructive, Not constructive
     13 sd_agreement  - Agreement throughout, Continual disagreement, Agreement to disagreement, Disagreement to agreement
     14 sd_type - Argumentative (back and forth), Flamewar (insulting), NA, Not persuasive, Off-topic/digression, Personal stories, Positive/respectful, Snarky/humorous
     15 sentiment - Mixed, Neutral, Negative, Positive
     16 tone - Controversial, Sarcastic, Informative
     17 commentagreement - Agree, Disagree, Adjunction Opinion
     18 topic - Offtopic with article, offtopic with conversation
     19 intendedaudience - Broadcast, Reply
     20 persuasiveness - Persuasive, Not persuasive


In [197]:
expert_annot[expert_annot.commentid == corpus.random_utterance().id]

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
260,68273,3,English Teacher Charged With Sexual Assault of...,http://www.insideedition.com/headlines/15888-e...,SGoApo4dLLXZvY83tDAX,00004n000000000000000000000000-51ca6e2b-243b-4...,1460819691,4.0,1.0,"russell, c'mon, you're better than that.",1460818837098-c6d3ff5e-f271-4f00-8fc5-935aa542...,Not constructive,Continual disagreement,"Snarky/humorous,Flamewar (insulting)",positive,"Sympathetic,Sarcastic",Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
775,68273,3,English Teacher Charged With Sexual Assault of...,http://www.insideedition.com/headlines/15888-e...,SGoApo4dLLXZvY83tDAX,00004n000000000000000000000000-51ca6e2b-243b-4...,1460819691,4.0,1.0,"russell, c'mon, you're better than that.",1460818837098-c6d3ff5e-f271-4f00-8fc5-935aa542...,Not constructive,Continual disagreement,"Argumentative (back and forth),Off-topic/digre...",neutral,Funny,Disagreement with commenter,"Off-topic with conversation,Off-topic with art...",Reply to a specific commenter,NaN
1764,68273,3,English Teacher Charged With Sexual Assault of...,http://www.insideedition.com/headlines/15888-e...,SGoApo4dLLXZvY83tDAX,00004n000000000000000000000000-51ca6e2b-243b-4...,1460819691,4.0,1.0,"russell, c'mon, you're better than that.",1460818837098-c6d3ff5e-f271-4f00-8fc5-935aa542...,Not constructive,Continual disagreement,Snarky/humorous,negative,NaN,Disagreement with commenter,NaN,Reply to a specific commenter,Not persuasive


In [198]:
corpus.get_utterance('00003n000000000000000000000000-19e62029-c449-412c-835a-5510b4ee8c65')

Utterance({'obj_type': 'utterance', '_owner': <convokit.model.corpus.Corpus object at 0x13ae25790>, 'meta': {}, '_id': '00003n000000000000000000000000-19e62029-c449-412c-835a-5510b4ee8c65', 'speaker': Speaker({'obj_type': 'speaker', '_owner': <convokit.model.corpus.Corpus object at 0x13ae25790>, 'meta': {}, '_id': 'r88CmTiCLmRLSBMVRQOT'}), 'conversation_id': '1461753212853-1effe7e2-6c15-494c-bf4a-68006ad1e05e', '_root': '1461753212853-1effe7e2-6c15-494c-bf4a-68006ad1e05e', 'reply_to': '00002n000000000000000000000000-e7f63a66-9fd9-4cb1-8edd-2bf17964c332', 'timestamp': 1461756899, 'text': "Women can have sex, too. As much or as little as they like and with as many or as few partners as they like. It's each persons responsibility to know what the boundaries of their current relationship is and to not go past them. If a woman is in a supposedly monogamous relationship, it is her responsibility not to sleep with other people. If a man is in a supposedly monogamous relationship, it is his re

In [199]:
corpus.get_utterance('53453')

KeyError: '53453'

In [200]:
multiple_annots = ['constructiveclass', 'sd_agreement', 'sd_type', 'sentiment', 'tone', 'commentagreement', 'topic',
                  'intendedaudience', 'persuasiveness']
for row in expert_annot.iterrows():
    try:
        data = row[1]
        utt = corpus.get_utterance(data.commentid)
        utt.meta['thumbs-up'] = data['thumbs-up']
        utt.meta['thumbs-down'] = data['thumbs-down']
        if multiple_annots[0] not in utt.meta:
            for k in multiple_annots:
                utt.meta[k] = []

        for k in multiple_annots:
            utt.meta[k].append(data[k])
    except KeyError:
        print("Skipped utterance " + data.commentid)

Skipped utterance 00009g000000000000000000000000-21de97e5-54dd-4dde-a162-b3e0d12164aa
Skipped utterance 00006g000000000000000000000000-6c9d9a6c-922c-4bbe-a962-00e949fbdb68
Skipped utterance 00008b000000000000000000000000-5ae4d4da-d334-4a66-99fd-f05873361f0d
Skipped utterance 00007n000000000000000000000000-e3096a96-0717-4267-a982-e05ccd54e67f
Skipped utterance 00007n000000000000000000000000-e3096a96-0717-4267-a982-e05ccd54e67f
Skipped utterance 00006g000000000000000000000000-6c9d9a6c-922c-4bbe-a962-00e949fbdb68
Skipped utterance 00009g000000000000000000000000-21de97e5-54dd-4dde-a162-b3e0d12164aa
Skipped utterance 00008b000000000000000000000000-5ae4d4da-d334-4a66-99fd-f05873361f0d
Skipped utterance 00007n000000000000000000000000-e3096a96-0717-4267-a982-e05ccd54e67f
Skipped utterance 00006g000000000000000000000000-6c9d9a6c-922c-4bbe-a962-00e949fbdb68
Skipped utterance 00009g000000000000000000000000-21de97e5-54dd-4dde-a162-b3e0d12164aa
Skipped utterance 00008b000000000000000000000000-5ae4d

In [205]:
# Removing nans
for utt in corpus.iter_utterances():
    for k in multiple_annots:
        utt.meta[k] = [v for v in utt.meta[k] if (isinstance(v, str))]

In [212]:
for utt in corpus.iter_utterances():
    if math.isnan(utt.meta['thumbs-up']):
        utt.meta['thumbs-up'] = None
    if math.isnan(utt.meta['thumbs-down']):
        utt.meta['thumbs-down'] = None

In [225]:
corpus.random_utterance().meta

{'thumbs-up': 1.0,
 'thumbs-down': 5.0,
 'constructiveclass': ['Constructive', 'Constructive', 'Not constructive'],
 'sd_agreement': ['Continual disagreement',
  'Initial disagreement converging to agreement'],
 'sd_type': ['Argumentative (back and forth),Snarky/humorous,Off-topic/digression,Positive/respectful',
  'Argumentative (back and forth),Off-topic/digression'],
 'sentiment': ['negative', 'negative', 'negative'],
 'tone': ['Mean', 'Controversial', 'Mean'],
 'commentagreement': ['Disagreement with commenter',
  'Disagreement with commenter',
  'Disagreement with commenter'],
 'topic': ['Off-topic with article'],
 'intendedaudience': ['Reply to a specific commenter',
  'Reply to a specific commenter',
  'Reply to a specific commenter'],
 'persuasiveness': ['Not persuasive', 'Not persuasive', 'Not persuasive']}

In [226]:
corpus.dump('annotated-ynacc', base_path='.')

In [227]:
corpus.print_summary_stats()

Number of Speakers: 6402
Number of Utterances: 9156
Number of Conversations: 1399


In [144]:
os.listdir()

['ydata-ynacc-v1_0_turk_annotations.tsv',
 'ydata-ynacc-v1_0_test-ids.txt',
 'ydata-ynacc-v1_0_IAC_annotations.tsv',
 'ydata-ynacc-v1_0_unlabeled_conversations.tsv',
 'ydata-ynacc-v1_0_dev-ids.txt',
 'WebscopeReadme.txt',
 'ydata-ynacc-v1_0_expert_annotations.tsv',
 'README.txt',
 'ydata-ynacc-v1_0_parentid_category.tsv',
 'ydata-ynacc-v1_0_unlabeled_conversations.csv',
 'ydata-ynacc-v1_0_train-ids.txt']

In [102]:
len(utts)

9156

In [57]:
expert_annot[expert_annot.sdid == 53971]

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness
0,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive
1,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,NaN,Constructive,NaN,Positive/respectful,mixed,NaN,NaN,Off-topic with article,Broadcast message / general audience,Not persuasive
2,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive
202,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,Agreement throughout,Positive/respectful,neutral,Informative,Adjunct opinion,NaN,Reply to a specific commenter,Persuasive
203,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,NaN,Constructive,Agreement throughout,Positive/respectful,positive,Sympathetic,NaN,NaN,Broadcast message / general audience,Not persuasive
204,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,Constructive,Agreement throughout,Positive/respectful,neutral,Informative,Agreement with commenter,NaN,Reply to a specific commenter,Not persuasive
1301,53971,2,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,rjrPtwH5oVVuQnEXX3hf,00003n000000000000000000000000-ed2ae6d0-32ac-4...,1459917444,NaN,NaN,"These things happen , Every job has its dangers.",1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,NaN,NaN,Positive/respectful,neutral,Controversial,Adjunct opinion,Off-topic with conversation,Broadcast message / general audience,NaN
1302,53971,0,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,VaW6HEsuOFUAIBqjw1k~,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,1459879464,1.0,NaN,Sad to hear such a bad thing. Very dangerous j...,NaN,NaN,NaN,Positive/respectful,neutral,"Sympathetic,NA",NaN,NaN,Broadcast message / general audience,NaN
1303,53971,1,Disneyland Worker Found Dead in Haunted Mansion,http://www.cosmopolitan.com/lifestyle/news/a56...,uwQePj970KaMZuW3~9Q9,00002n000000000000000000000000-1c30b878-b717-4...,1459881644,NaN,NaN,Yes..because too many houses in EU look like t...,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...,NaN,NaN,Positive/respectful,neutral,Informative,Adjunct opinion,Off-topic with article,Reply to a specific commenter,NaN


In [49]:
utts = []
ctr = 0
for row in expert_annot.iterrows():
    data = row[1]
    parent_id = validate_data(data.commentid)
        
    if data.commentindex == 0:
        reply_to = None
    else:
        pass
    utt = Utterance(id=data.commentid, speaker=data.guid, timestamp=data.timestamp, 
                    conversation_id = parent_id,
                    text=data.text, reply_to=parent_id)
    utts.append(utt)
    print(data.parentid)
    print(data)
    break
#     utts.append(Utterance(id=data.comment_id, conversation_id=data.parentid if data.p))
    if ctr == 2:
        break
    ctr += 1

1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249
sdid                                                             53971
commentindex                                                         2
headline               Disneyland Worker Found Dead in Haunted Mansion
url                  http://www.cosmopolitan.com/lifestyle/news/a56...
guid                                              rjrPtwH5oVVuQnEXX3hf
commentid            00003n000000000000000000000000-ed2ae6d0-32ac-4...
timestamp                                                   1459917444
thumbs-up                                                          NaN
thumbs-down                                                        NaN
text                  These things happen , Every job has its dangers.
parentid             1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b...
constructiveclass                                         Constructive
sd_agreement                                                       NaN
sd_type                   

In [45]:
utts

[Utterance({'obj_type': 'utterance', '_owner': None, 'meta': {}, '_id': '1461008271435-3836a3e7-6f93-4385-8386-686761d646fa', 'speaker': 'OvVPMSRmSYCVhryYrhRj', 'conversation_id': nan, '_root': nan, 'reply_to': nan, 'timestamp': 1461008271.0, 'text': "They are taking all the great shows off or ending the characters that made it great. I don't understand what they are thinking, we did away with Dish bc they kept taking shows of and raising the price then you have MTV doing away with Finding Carter, AE doing away with Appalachian outlaw.  We just have regular TV and now I guess I won't be watching Castle! I loved all these shows and it's like they don't want you watching anything good. If people aren't naked or parting then they don't want it. I think I'm gonna just go back to buying movies again. At least they can't be canceled in the middle of it."}),
 Utterance({'obj_type': 'utterance', '_owner': None, 'meta': {}, '_id': '00002g000000000000000000000000-a1b3b015-db5b-428e-8fb5-3c44897c

In [34]:
basic_data

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid
0,129908,0,Stana Katic Out at 'Castle' as Part of Season ...,https://tvline.com/2016/04/18/stana-katic-leav...,OvVPMSRmSYCVhryYrhRj,1461008271435-3836a3e7-6f93-4385-8386-686761d6...,1.461008e+09,44.0,0.0,They are taking all the great shows off or end...,NaN
1,129908,1,Stana Katic Out at 'Castle' as Part of Season ...,https://tvline.com/2016/04/18/stana-katic-leav...,LKaND3iPvrz7ISWIR0pK,00002g000000000000000000000000-a1b3b015-db5b-4...,1.461020e+09,0.0,0.0,"The prior seasons are in reruns, so you can st...",1461008271435-3836a3e7-6f93-4385-8386-686761d6...
2,129908,2,Stana Katic Out at 'Castle' as Part of Season ...,https://tvline.com/2016/04/18/stana-katic-leav...,auP242ond1glYjOtoeKr,00003g000000000000000000000000-e57e0b92-87ec-4...,1.461031e+09,5.0,1.0,They are thinking that it is much cheaper to p...,1461008271435-3836a3e7-6f93-4385-8386-686761d6...
3,288,0,Math Teacher Marries 18-Year-Old Former Studen...,http://www.cosmopolitan.com/sex-love/news/a564...,adNyuz2Nq9eyBHIb9v9R,1460146291971-5b920c10-ae4b-464f-ba2d-2899b3bd...,1.460146e+09,11.0,4.0,"To the American mind, a child is a magical, in...",NaN
4,288,1,Math Teacher Marries 18-Year-Old Former Studen...,http://www.cosmopolitan.com/sex-love/news/a564...,PhUrLGRcIBtfbzbArSRh,00002g000000000000000000000000-57d5448a-4fe7-4...,1.460147e+09,3.0,6.0,Haven't you already posted this comment? Move on.,1460146291971-5b920c10-ae4b-464f-ba2d-2899b3bd...
5,288,2,Math Teacher Marries 18-Year-Old Former Studen...,http://www.cosmopolitan.com/sex-love/news/a564...,PMejQtxEb34eer6P0rKJ,00003b000000000000000000000000-6cdb2380-433e-4...,1.460148e+09,4.0,2.0,What the hell do Hobbits and Lord of the Rings...,1460146291971-5b920c10-ae4b-464f-ba2d-2899b3bd...
6,288,3,Math Teacher Marries 18-Year-Old Former Studen...,http://www.cosmopolitan.com/sex-love/news/a564...,QeKz~WalRAx1U38gTq9s,00004b000000000000000000000000-d7dd3133-ca35-4...,1.460148e+09,3.0,1.0,@USERNAME sex with a 17 year old is hardly wha...,1460146291971-5b920c10-ae4b-464f-ba2d-2899b3bd...
7,288,4,Math Teacher Marries 18-Year-Old Former Studen...,http://www.cosmopolitan.com/sex-love/news/a564...,Xu3357VGxXTpNrA6Oj66,00005g000000000000000000000000-708efeaf-4e11-4...,1.460150e+09,1.0,4.0,Nate had a girlfriend once but she left him fo...,1460146291971-5b920c10-ae4b-464f-ba2d-2899b3bd...
8,61038,0,AP-GfK Poll: Clinton has edge over Trump on ra...,http://ap-org253133660-yahoopartner.tumblr.com...,uvSlz1DnB6cMKj~GXD7M,1460394808008-5551a682-4d8a-4a50-9453-6b841f86...,1.460395e+09,44.0,5.0,American's trust Clinton more than Trump? Are...,NaN
9,61038,1,AP-GfK Poll: Clinton has edge over Trump on ra...,http://ap-org253133660-yahoopartner.tumblr.com...,8bV5sz~i4NJOAIjmI5dg,00002b000000000000000000000000-2821ae6b-8554-4...,1.460395e+09,6.0,1.0,"Yep, proof that the idiots outnumber the sane,...",1460394808008-5551a682-4d8a-4a50-9453-6b841f86...


In [9]:
turk_annot = pd.read_csv('ydata-ynacc-v1_0_turk_annotations.tsv', sep='\t')

In [10]:
turk_annot

,sdid,commentindex,headline,url,guid,commentid,timestamp,thumbs-up,thumbs-down,text,parentid,constructiveclass,sd_agreement,sd_type
0,50735,0,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,QcJlZUh9ahUFn0EHRgaS,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,1459625991,21,13,"OMG, I should shoot myself for voting for her ...",1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
1,50735,1,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,IYp4~4xH1otqco8_0iDq,00002g000000000000000000000000-1d5d6112-78c5-4...,1459626067,4,5,yep,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
2,50735,2,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,T_rh7XPzhe8SgWNLJx1N,00003n000000000000000000000000-dd89dec7-63cd-4...,1459626261,1,1,you got it what a human or animal,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
3,50735,3,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,OgLSacyNcC9ECBmF5yWs,00004b000000000000000000000000-e377ed85-e972-4...,1459626297,2,2,just wing yourself..a flesh wound will be fine,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
4,50735,4,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,aNYk4ScmUVFLZ_NG_0RF,00005g000000000000000000000000-b6a381b1-8c11-4...,1459626334,0,2,Nope!,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
5,50735,5,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,aNYk4ScmUVFLZ_NG_0RF,00006g000000000000000000000000-06f25b75-8384-4...,1459626382,0,4,@USERNAME stop reading the tabloids! Go Trump!,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
6,50735,6,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,tteUO7luDRKjoWfaXia2,00008g000000000000000000000000-a0444513-1d2e-4...,1459629631,0,0,God love you Please forgive them for they alw...,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Continual disagreement,Flamewar
7,50735,0,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,QcJlZUh9ahUFn0EHRgaS,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,1459625991,21,13,"OMG, I should shoot myself for voting for her ...",1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Initial agreement,Snarky/Humorous|Positive/respectful|Off-Topic ...
8,50735,1,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,IYp4~4xH1otqco8_0iDq,00002g000000000000000000000000-1d5d6112-78c5-4...,1459626067,4,5,yep,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Initial agreement,Snarky/Humorous|Positive/respectful|Off-Topic ...
9,50735,2,Sarah Palin's Latest Anti-Immigrant Comments C...,http://mic.com/articles/139658/sarah-palin-s-l...,T_rh7XPzhe8SgWNLJx1N,00003n000000000000000000000000-dd89dec7-63cd-4...,1459626261,1,1,you got it what a human or animal,1459625991553-ca679f42-7072-4ed4-a22b-9271437a...,Not constructive,Initial agreement,Snarky/Humorous|Positive/respectful|Off-Topic ...
